In [1]:
import requests
import json

BASE_URL = "http://localhost:8000"
print("BASE_URL =", BASE_URL)


BASE_URL = http://localhost:8000


## 1️⃣ GET /tags – List All Tags

Fetch all tags. Should be empty at the beginning unless tags already exist.


In [2]:
url = f"{BASE_URL}/tags/"
res = requests.get(url)

print("Status:", res.status_code)
tags_list = res.json()
print("Existing tags:", len(tags_list))
print(json.dumps(tags_list[:5], indent=2))


Status: 200
Existing tags: 0
[]


## 2️⃣ POST /tags – Create New Tags

We will create a batch of tags and verify successful creation.


In [3]:
test_tags = [
    "hiking", "scenic", "family-friendly", "difficult", "sunset", "forest",
    "urban", "photography", "historical", "panorama"
]

created_tag_ids = []

for tag_name in test_tags:
    payload = {"name": tag_name}
    url = f"{BASE_URL}/tags/"

    res = requests.post(url, json=payload)
    print(f"Creating '{tag_name}' → Status {res.status_code}")

    if res.status_code == 200:
        created_tag_ids.append(res.json().get("id"))

created_tag_ids[:5]


Creating 'hiking' → Status 200
Creating 'scenic' → Status 200
Creating 'family-friendly' → Status 200
Creating 'difficult' → Status 200
Creating 'sunset' → Status 200
Creating 'forest' → Status 200
Creating 'urban' → Status 200
Creating 'photography' → Status 200
Creating 'historical' → Status 200
Creating 'panorama' → Status 200


[1, 2, 3, 4, 5]

## 3️⃣ Attempting Duplicate Tag Creation (Should Fail)

Duplicate tag names should return a 400 error.


In [4]:
duplicate_payload = {"name": test_tags[0]}  # take first tag
url = f"{BASE_URL}/tags/"

res = requests.post(url, json=duplicate_payload)

print("Status:", res.status_code)
print("Response:")
print(json.dumps(res.json(), indent=2))


Status: 400
Response:
{
  "detail": "Tag already exists"
}


## 4️⃣ GET /tags – Verify Tag List After Creation

List tags again to verify the number of existing tags increased.


In [5]:
url = f"{BASE_URL}/tags/"
res = requests.get(url)

print("Status:", res.status_code)
tags_list = res.json()
print("Total tags now:", len(tags_list))
print(json.dumps(tags_list[:8], indent=2))


Status: 200
Total tags now: 10
[
  {
    "name": "difficult",
    "id": 4
  },
  {
    "name": "family-friendly",
    "id": 3
  },
  {
    "name": "forest",
    "id": 6
  },
  {
    "name": "hiking",
    "id": 1
  },
  {
    "name": "historical",
    "id": 9
  },
  {
    "name": "panorama",
    "id": 10
  },
  {
    "name": "photography",
    "id": 8
  },
  {
    "name": "scenic",
    "id": 2
  }
]


## 5️⃣ DELETE /tags/{id} – Delete a Tag

We delete the first created tag.


In [6]:
if created_tag_ids:
    tag_id_to_delete = created_tag_ids[0]
    url = f"{BASE_URL}/tags/{tag_id_to_delete}"

    res = requests.delete(url)
    print("Deleting tag_id:", tag_id_to_delete)
    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))
else:
    print("No created tags available to delete.")


Deleting tag_id: 1
Status: 200
{
  "message": "Tag deleted successfully"
}


## 6️⃣ Verify Deletion by Listing Tags Again

Ensure the removed tag no longer appears.


In [ ]:
url = f"{BASE_URL}/tags/"
res = requests.get(url)
print("Status:", res.status_code)
print("Tags after deletion:")
print(json.dumps(res.json()[:8], indent=2))


Status: 200
Tags after deletion:
[
  {
    "name": "difficult",
    "id": 4
  },
  {
    "name": "family-friendly",
    "id": 3
  },
  {
    "name": "forest",
    "id": 6
  },
  {
    "name": "historical",
    "id": 9
  },
  {
    "name": "panorama",
    "id": 10
  },
  {
    "name": "photography",
    "id": 8
  },
  {
    "name": "scenic",
    "id": 2
  },
  {
    "name": "sunset",
    "id": 5
  }
]


## 7️⃣ Fetch Random Locations for Tag Testing

Get some existing locations from the database to test tag assignment.

In [2]:
import random

# Fetch all locations
url = f"{BASE_URL}/locations/"
res = requests.get(url)

print("Status:", res.status_code)
all_locations = res.json()

if all_locations:
    # Pick 2-3 random locations for testing
    test_locations = random.sample(all_locations, min(3, len(all_locations)))
    test_location_ids = [loc["id"] for loc in test_locations]
    
    print(f"Selected {len(test_locations)} locations for testing:")
    for loc in test_locations:
        print(f"  - {loc['name']} (ID: {loc['id']})")
else:
    test_location_ids = []
    print("No locations found in database. Create some locations first.")

test_location_ids

Status: 200
Selected 3 locations for testing:
  - Sharp Peak (ID: 4c51d729-73f0-436e-a64b-01ab20878447)
  - Tung Chung Valley (ID: 6b661402-2797-4753-9552-8d87b1e831c0)
  - Pat Sin Leng (ID: 0ad0c807-ce5b-41cb-b09d-95bff4472a7c)


['4c51d729-73f0-436e-a64b-01ab20878447',
 '6b661402-2797-4753-9552-8d87b1e831c0',
 '0ad0c807-ce5b-41cb-b09d-95bff4472a7c']

## 8️⃣ POST /locations/{location_id}/tags – Add Tags to Location

Add multiple tags to a location. The endpoint accepts a list of tag names.

In [3]:
if not test_location_ids:
    print("No test locations available. Run previous cell first.")
else:
    # Add tags to the first test location
    location_id = test_location_ids[0]
    
    # Use tags we created earlier
    tags_to_add = ["hiking", "scenic", "sunset"]
    
    payload = {"tags": tags_to_add}
    url = f"{BASE_URL}/locations/{location_id}/tags"
    
    res = requests.post(url, json=payload)
    
    print(f"Adding tags to location {location_id}")
    print("Status:", res.status_code)
    print("\nResponse:")
    print(json.dumps(res.json(), indent=2))

Adding tags to location 4c51d729-73f0-436e-a64b-01ab20878447
Status: 200

Response:
{
  "added_tags": [
    {
      "name": "hiking",
      "id": 11
    },
    {
      "name": "scenic",
      "id": 2
    },
    {
      "name": "sunset",
      "id": 5
    }
  ]
}


## 9️⃣ Verify Location Has Tags – GET /locations/{location_id}

Fetch the location details to see the attached tags.

In [4]:
if not test_location_ids:
    print("No test locations available.")
else:
    location_id = test_location_ids[0]
    
    url = f"{BASE_URL}/locations/{location_id}"
    res = requests.get(url)
    
    print("Status:", res.status_code)
    location_detail = res.json()
    
    print(f"\nLocation: {location_detail['location']['name']}")
    print(f"Tags attached: {len(location_detail.get('tags', []))}")
    print("\nTags:")
    for tag in location_detail.get('tags', []):
        print(f"  - {tag['name']} (ID: {tag['id']})")

Status: 200

Location: Sharp Peak
Tags attached: 3

Tags:
  - scenic (ID: 2)
  - sunset (ID: 5)
  - hiking (ID: 11)


## 🔟 Add Different Tags to Another Location

Test adding tags to a second location with different tag combinations.

In [5]:
if len(test_location_ids) < 2:
    print("Need at least 2 test locations. Skipping this test.")
else:
    # Add different tags to second location
    location_id = test_location_ids[1]
    
    tags_to_add = ["urban", "photography", "historical"]
    
    payload = {"tags": tags_to_add}
    url = f"{BASE_URL}/locations/{location_id}/tags"
    
    res = requests.post(url, json=payload)
    
    print(f"Adding tags to location {location_id}")
    print("Status:", res.status_code)
    print("\nAdded tags:")
    for tag in res.json().get("added_tags", []):
        print(f"  - {tag['name']}")

Adding tags to location 6b661402-2797-4753-9552-8d87b1e831c0
Status: 200

Added tags:
  - urban
  - photography
  - historical


## 1️⃣1️⃣ DELETE /locations/{location_id}/tags/{tag_id} – Remove Tag from Location

Remove a specific tag from a location by tag ID.

In [6]:
if not test_location_ids:
    print("No test locations available.")
else:
    location_id = test_location_ids[0]
    
    # First, get the location's tags
    url = f"{BASE_URL}/locations/{location_id}"
    res = requests.get(url)
    location_detail = res.json()
    
    tags = location_detail.get('tags', [])
    
    if not tags:
        print(f"Location {location_id} has no tags to remove.")
    else:
        # Remove the first tag
        tag_to_remove = tags[0]
        tag_id = tag_to_remove['id']
        tag_name = tag_to_remove['name']
        
        url = f"{BASE_URL}/locations/{location_id}/tags/{tag_id}"
        res = requests.delete(url)
        
        print(f"Removing tag '{tag_name}' (ID: {tag_id}) from location")
        print("Status:", res.status_code)
        print(json.dumps(res.json(), indent=2))

Removing tag 'scenic' (ID: 2) from location
Status: 200
{
  "message": "Tag removed"
}


## 1️⃣2️⃣ Verify Tag Removal – Check Location Tags Again

Confirm that the tag was successfully removed from the location.

In [7]:
if not test_location_ids:
    print("No test locations available.")
else:
    location_id = test_location_ids[0]
    
    url = f"{BASE_URL}/locations/{location_id}"
    res = requests.get(url)
    
    print("Status:", res.status_code)
    location_detail = res.json()
    
    print(f"\nLocation: {location_detail['location']['name']}")
    print(f"Tags now attached: {len(location_detail.get('tags', []))}")
    print("\nRemaining tags:")
    for tag in location_detail.get('tags', []):
        print(f"  - {tag['name']} (ID: {tag['id']})")

Status: 200

Location: Sharp Peak
Tags now attached: 2

Remaining tags:
  - sunset (ID: 5)
  - hiking (ID: 11)


## 1️⃣3️⃣ Filter Locations by Tags – GET /locations/by-tags

Test the new endpoint that filters locations by one or more tags.

In [9]:
# Test 1: Filter by single tag
print("=== Test 1: Filter by single tag 'hiking' ===")
url = f"{BASE_URL}/locations/by-tags?tags=hiking"
res = requests.get(url)

print("Status:", res.status_code)
hiking_locations = res.json()
print(f"Found {len(hiking_locations)} locations with 'hiking' tag")

if hiking_locations:
    for loc in hiking_locations:
        print(f"  Name: {loc['location']['name']}")
        print(f"  Tags: {[t['name'] for t in loc['tags']]}")

print("\n" + "="*60 + "\n")

# Test 2: Filter by multiple tags (ANY match)
print("=== Test 2: Filter by multiple tags (ANY: hiking OR scenic) ===")
url = f"{BASE_URL}/locations/by-tags?tags=hiking,scenic&match_all=false"
res = requests.get(url)

print("Status:", res.status_code)
any_match_locations = res.json()
print(f"Found {len(any_match_locations)} locations with 'hiking' OR 'scenic' tag")

print("\n" + "="*60 + "\n")

# Test 3: Filter by multiple tags (ALL match)
print("=== Test 3: Filter by multiple tags (ALL: hiking AND scenic) ===")
url = f"{BASE_URL}/locations/by-tags?tags=hiking,scenic&match_all=true"
res = requests.get(url)

print("Status:", res.status_code)
all_match_locations = res.json()
print(f"Found {len(all_match_locations)} locations with BOTH 'hiking' AND 'scenic' tags")

if all_match_locations:
    print("\nLocations with both tags:")
    for loc in all_match_locations[:3]:
        print(f"  - {loc['location']['name']}: {[t['name'] for t in loc['tags']]}")

=== Test 1: Filter by single tag 'hiking' ===
Status: 200
Found 2 locations with 'hiking' tag
  Name: Sharp Peak
  Tags: ['hiking', 'sunset']
  Name: Updated Test Location
  Tags: ['hiking', 'scenic', 'sunset']


=== Test 2: Filter by multiple tags (ANY: hiking OR scenic) ===
Status: 200
Found 2 locations with 'hiking' OR 'scenic' tag


=== Test 3: Filter by multiple tags (ALL: hiking AND scenic) ===
Status: 200
Found 1 locations with BOTH 'hiking' AND 'scenic' tags

Locations with both tags:
  - Updated Test Location: ['hiking', 'scenic', 'sunset']


## 1️⃣4️⃣ Test Non-Existent Tag Filter

Test what happens when filtering by tags that don't exist or no locations match.

In [10]:
# Filter by a tag that doesn't exist
url = f"{BASE_URL}/locations/by-tags?tags=nonexistent-tag-12345"
res = requests.get(url)

print("Status:", res.status_code)
print("Response:", res.json())
print("\nExpected: Empty list [] when no locations match the tags")

Status: 200
Response: []

Expected: Empty list [] when no locations match the tags
